# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - należy zaimplementować w tym celu prostą wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # <<< HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [9]:
from typing import Tuple, List
import math, random, copy

from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove

Wielkość planszy

In [10]:
ROW_COUNT = 6
COLUMN_COUNT = 7
WINDOW_LENGTH = 4

In [11]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour, depth):
        self.game = game
        self.depth = depth
        self._current_player = None

    def get_current_player(self):
        return self._current_player

    def set_current_player(self, new_current_player):
        self._current_player = new_current_player

    def evaluate_position(self, player: Player)->float:
        score = 0

        # Score horizontal positions
        for r in range(ROW_COUNT):
            row_array = [self.game.state.fields[i][r] for i in range(COLUMN_COUNT)]
            #for some weirds reasons column adn row indexes are switched in game.state.fields
            for c in range(COLUMN_COUNT - 3):
                # Create a horizontal window of 4
                window = row_array[c:c + WINDOW_LENGTH]
                score += self.evaluate_window(window, player)

        # Score vertical positions
        for c in range(COLUMN_COUNT):
            col_array = [self.game.state.fields[c][i] for i in range(ROW_COUNT)]
            #for some weirds reasons column and row indexes are switched in game.state.fields
            for r in range(ROW_COUNT - 3):
                # Create a vertical window of 4
                window = col_array[r:r + WINDOW_LENGTH]
                score += self.evaluate_window(window, player)

        # Score positive diagonals
        for r in range(ROW_COUNT - 3):
            for c in range(COLUMN_COUNT - 3):
                # Create a positive diagonal window of 4
                window = [self.game.state.fields[c + i][r + i] for i in range(WINDOW_LENGTH)]
                #for some weirds reasons column and row indexes are switched in game.state.fields
                score += self.evaluate_window(window, player)

        # Score negative diagonals
        for r in range(ROW_COUNT - 3):
            for c in range(COLUMN_COUNT - 3):
                # Create a negative diagonal window of 4
                window = [self.game.state.fields[c + i][r + 3 - i] for i in range(WINDOW_LENGTH)]
                #for some weirds reasons column adn row indexes are switched in game.state.fields
                score += self.evaluate_window(window, player)
        return score

    def evaluate_window(self, window, player: Player)->float:
        score = 0
        if player.char == self.game.first_player.char:
            current_player = self.game.first_player
            opposite_player = self.game.second_player
        else:
            current_player = self.game.second_player
            opposite_player = self.game.first_player

        if window.count(current_player) == 4:
            score += 100
        elif window.count(current_player) == 3 and window.count(None) == 1:
            score += 15
        elif window.count(current_player) == 2 and window.count(None) == 2:
            score += 7
        elif window.count(current_player) == 1 and window.count(None) == 3:
            score += 2

        if window.count(opposite_player) == 2 and window.count(None) == 2:
            score -= 2
        elif window.count(opposite_player) == 3 and window.count(None) == 1:
            score -= 10
        elif window.count(opposite_player) == 4:
            score -= 100
        return score


    def get_best_move(self)->int:
        pass

    def is_valid_move(self, col_index:int)->bool:
        return col_index in self.get_valid_moves()

    def get_valid_moves(self)->List[int]:
        return [move.column for move in self.game.get_moves()]

    def minimax(self, depth, alpha:float, beta:float, is_maximizing_player:bool)-> Tuple[int, float]:
        valid_moves = self.get_valid_moves()
        is_finished = self.game.is_finished()

        if depth == 0 or is_finished:
            if is_finished:
                if self.game.get_winner() == self.game.first_player:
                    return None, 100000
                elif self.game.get_winner() == self.game.second_player:
                    return None, -100000
                else:
                    return None, 0
            else: # Depth is zero
                # if is_maximizing_player:
                #     return None, self.evaluate_position(self.game.state.get_current_player())
                # else:
                #     return None, self.evaluate_position(self.game.state.get_players()[1])
                return None, self.evaluate_position(self.get_current_player())
        if is_maximizing_player:
            value = -math.inf
            # column = random.choice(valid_moves)
            for col in valid_moves:
                game_copy = copy.deepcopy(self.game)
                if depth == self.depth:
                    if self.get_current_player().char == 'a':
                        # game_copy = copy.deepcopy(self.game)
                        pass
                    else:
                        game_copy.first_player = self.game.second_player
                        game_copy.second_player = self.game.first_player
                else:
                    game_copy.first_player = self.game.second_player
                    game_copy.second_player = self.game.first_player
                # game_copy.state.fields = self.game.state.fields
                game_copy.make_move(ConnectFourMove(col)) # Gotta check if _current_player is valid
                copy_minimax = MinMaxSolver(game_copy, self.depth)
                new_score = copy_minimax.minimax(depth-1, alpha, beta, False)[1]

                if new_score > value:
                    value = new_score
                    column = col
                alpha = max(alpha, value)
                if alpha >= beta:
                    break
                    # return column, alpha
            # return column, value

        else: # Minimizing player
            value = math.inf
            # column = random.choice(valid_moves)
            for col in valid_moves:
                game_copy = copy.deepcopy(self.game)
                # game_copy.state.fields = self.game.state.fields
                game_copy.make_move(ConnectFourMove(col)) # Gotta check if _current_player is valid
                copy_minimax = MinMaxSolver(game_copy, self.depth)
                new_score = copy_minimax.minimax(depth-1, alpha, beta, True)[1]

                if new_score < value:
                    value = new_score
                    column = col
                beta = min(beta, value)
                if alpha >= beta:
                    break
            # return column, value

        return column, value


Rozgrywka

In [12]:
p1 = Player("a")
p2 = Player("b")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
# game.make_move(ConnectFourMove(6))
# game.make_move(ConnectFourMove(2))
# game.make_move(ConnectFourMove(6))
# game.make_move(ConnectFourMove(3))
# game.make_move(ConnectFourMove(2))
print(game)

depth = 7
mini_max_solver = MinMaxSolver(game, depth)
iteration = 0
while not game.is_finished():
    column, value = mini_max_solver.minimax(depth, -math.inf, math.inf, True)
    game.make_move(ConnectFourMove(column))
    iteration += 1
    if iteration == 20:
        print(iteration)
    print(iteration)
    print(game)
print(game.get_winner().char)

# column, value = mini_max_solver.minimax(4, -math.inf, math.inf, True)
# game.make_move(ConnectFourMove(column))
# print(game)

# print(game.get_winner().char)
# print(str(game.get_winner().char))

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]


AttributeError: 'NoneType' object has no attribute 'char'